## Part 4: Fraud

In [1]:
%run "../scripts/outlier.py" "../scripts/paths.json"

/Users/ahirve/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 14:19:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/10/07 14:19:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


22/10/07 14:20:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
 %run "../scripts/fraud_feature.py" "../scripts/paths.json"

In [41]:
cat = spark.read.parquet('/Users/ahirve/Desktop/ads_bnpl/data/tables/transactions_with_fraud_rates.parquet')
cat.limit(5)

cat = cat.withColumnRenamed("merchant_abn","sus_merchant_abn")
full_data = spark.read.parquet('../data/tables/full_join.parquet')

full_data.createOrReplaceTempView("temp")

cat.createOrReplaceTempView("temp2")

full_data_with_fraud = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.sus_merchant_abn
""")

full_data_with_fraud.write.parquet('../data/tables/full_dataset_with_fraud_rates.parquet')


AnalysisException: path file:/Users/ahirve/Desktop/ads_bnpl/data/tables/full_dataset_with_fraud_rates.parquet already exists.

In [53]:

full_data_with_fraud = full_data_with_fraud.select("merchant_name", "merchant_abn","take_rate","revenue_levels","name","address","state","gender","dollar_value","order_id","order_datetime","user_id","consumer_id","postcodes","int_sa2","SA2_code","SA2_name","income_2018-2019","total_males","total_females","total_persons","state_code","state_name","population_2020","population_2021","categories")

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021,fraud_probability_consumer,fraud_probability_merchant,sus_merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6,is_fraud
Eleifend Nec Inco...,21532935983,"cable, satellite,...",5.58,a,Tamara Stewart,352 Jessica Summit,VIC,Female,21532935983,71.1148505207073,5fbb2316-39b7-43b...,2021-08-21,13842,30,3612,216011410,216011410,Rushworth,94881066,2161,2033,4193,2,Victoria,4121,4166,0.01,0.01,21532935983,"cable, satellite,...",a,13.493546079328608,13518,946992,13518,2021-12-22,40.479913479546646,null,null,null,522,Eleifend Nec Inco...,"([cable, satellit...",21532935983,cable satellite a...,4,Electronics,0
Eleifend Nec Inco...,21532935983,"cable, satellite,...",5.58,a,Tamara Stewart,352 Jessica Summit,VIC,Female,21532935983,71.1148505207073,5fbb2316-39b7-43b...,2021-08-21,13842,30,3612,216011410,216011410,Rushworth,94881066,2161,2033,4193,2,Victoria,4121,4166,0.01,0.01,21532935983,"cable, satellite,...",a,13.493546079328608,13518,946992,13518,2021-11-04,48.34257215824094,null,null,null,522,Eleifend Nec Inco...,"([cable, satellit...",21532935983,cable satellite a...,4,Electronics,0
Eleifend Nec Inco...,21532935983,"cable, satellite,...",5.58,a,Tamara Stewart,352 Jessica Summit,VIC,Female,21532935983,71.1148505207073,5fbb2316-39b7-43b...,2021-08-21,13842,30,3612,216011410,216011410,Rushworth,94881066,2161,2033,4193,2,Victoria,4121,4166,0.01,0.01,21532935983,"cable, satellite,...",a,22.155549246983117,1840,1267467,1840,2022-01-24,10.44073130573142,null,null,null,522,Eleifend Nec Inco...,"([cable, satellit...",21532935983,cable satellite a...,4,Electronics,0
Eleifend Nec Inco...,21532935983,"cable, satellite,...",5.58,a,Tamara Stewart,352 Jessica Summit,VIC,Female,21532935983,71.1148505207073,5fbb2316-39b7-43b...,2021-08-21,13842,30,3612,216011410,216011410,Rushworth,94881066,2161,2033,4193,2,Victoria,4121,4166,0.01,0.01,21532935983,"cable, satellite,...",a,23.175321834450585,14064,1444609,14064,2022-02-24,10.982822103322263,null,null,null,522,Eleifend Nec Inco...,"([cable, satellit...",21532935983,cable satellite a...,4,Electronics,0
Eleifend Nec Inco...,21532935983,"cable, satellite,...",5.58,a,Tamara Stewart,352 Jessica Summit,VIC,Female,21532935983,71.1148505207073,5fbb2316-39b7-43b...,2021-08-21,13842,30,3612,216011410,216011410,Rushworth,94881066,2161,2033,4193,2,Victoria,4121,4166,0.01,0.01,21532935983,"cable, satellite,...",a,23.175321834450585,14064,1444609,14064,2021-09-13,11.883808291360957,null,null,null,522,Eleifend Nec Inco...,"([cable, satellit...",21532935983,cable satellite a...,4,Electronics,0


In [51]:
model_with_fraud.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6,is_fraud
63290521567,artist supply and...,a,55.6054944945403,667,1549,667,2021-09-09,17.802875300207035,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY,0
63290521567,artist supply and...,a,55.6054944945403,667,1549,667,2021-10-04,9.674041120495586,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY,0
24852446429,florists supplies...,c,44.50284528982194,667,1549,667,2021-09-09,17.802875300207035,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture,0
24852446429,florists supplies...,c,44.50284528982194,667,1549,667,2021-10-04,9.674041120495586,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture,0
68216911708,"computers, comput...",c,18.476243072950304,667,1549,667,2021-09-09,17.802875300207035,null,null,null,2620,Placerat Eget Ven...,"[[computers, comp...",68216911708,computers compute...,4,Electronics,0


In [48]:
model = spark.read.parquet("../data/tables/transactions_with_fraud_rates.parquet")
model.limit(5)

merchant_abn,average fraud rate per merchant
83412691377,0.0
48214071373,0.0
38700038932,0.0
41956465747,0.0
73256306726,0.0


In [5]:
#Loading consumer fraud data: 
fraud_consumer = pd.read_csv('../data/tables/consumer_fraud_probability.csv')
fraud_consumer = spark.createDataFrame(fraud_consumer) 

#Loading merchant fraud data: 
fraud_merchant = pd.read_csv('../data/tables/merchant_fraud_probability.csv')
fraud_merchant = spark.createDataFrame(fraud_merchant) 


In [10]:
full_data = spark.read.parquet('../data/tables/full_join.parquet')

In [11]:
full_data.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021,fraud_probability_consumer,fraud_probability_merchant
Dui Nec Corporation,19933438190,tent and awning s...,2.22,c,Brian Wright,761 Bennett Mountain,TAS,Male,19933438190,3.434666831614868,28ccb7e6-ebe7-496...,2021-06-17,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563,0.01,0.01
Vel Est Tempor LLP,62694031334,"computers, comput...",5.76,a,Brian Wright,761 Bennett Mountain,TAS,Male,62694031334,30.583567099986173,d5902566-aa14-4e4...,2021-05-07,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563,0.01,0.01
Aliquam Nisl Corp...,55325186693,"gift, card, novel...",4.44,b,Brian Wright,761 Bennett Mountain,TAS,Male,55325186693,68.91163123686927,234aedaa-1657-466...,2021-07-20,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563,0.01,0.01
Urna Incorporated,78905182348,"watch, clock, and...",4.65,b,Brian Wright,761 Bennett Mountain,TAS,Male,78905182348,15.129648652621713,51d13e41-34a5-496...,2021-06-23,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563,0.01,0.01
Purus Gravida Sag...,21772962346,florists supplies...,6.63,a,Brian Wright,761 Bennett Mountain,TAS,Male,21772962346,0.7692714085929745,af89a162-38c7-48a...,2021-04-21,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563,0.01,0.01


The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [7]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

34864


user_id_1,order_datetime_1,cons_fraud_prob
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508036


Probability of fraud for high revenue merchants...?

In [8]:
print(fraud_merchant.count())
fraud_merchant = fraud_merchant.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchant.limit(5)

114


merchant_abn_1,order_datetime,merch_fraud_prob
19492220327,2021-11-28,44.40365864749536
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [ ]:
#reading in tagged merchant data 


In [12]:

merchant_data = full_data.select("merchant_abn","revenue_levels","dollar_value","user_id","consumer_id","postcodes")

In [13]:
merchant_data.limit(5)

merchant_abn,revenue_levels,dollar_value,user_id,consumer_id,postcodes
19933438190,c,3.434666831614868,2582,743,7016
62694031334,a,30.583567099986173,2582,743,7016
55325186693,b,68.91163123686927,2582,743,7016
78905182348,b,15.129648652621713,2582,743,7016
21772962346,a,0.7692714085929745,2582,743,7016


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [14]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

LEFT JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")


In [15]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchant.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")

In [16]:
# #Changing all null probability values to 0.01: 

# MIN_P = 0.01
# final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [17]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [18]:
# Reading in segments 
segments_data = spark.read.csv('../data/curated/tagged_merchants.csv')

In [19]:
segments_data.limit(5)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
null,name,tags,merchant_abn,cleaned_tags,store_type,category
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY


In [20]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY
4,Enim Condimentum PC,([music shops - m...,10192359162,music shops musi...,3,"Books, Stationary..."
5,Fusce Company,"[(gift, card, nov...",10206519221,gift card novelty...,4,Electronics
6,Aliquam Enim Inco...,"[(computers, comP...",10255988167,computers compute...,4,Electronics
7,Ipsum Primis Ltd,"[[watch, clock, a...",10264435225,watch clock and j...,1,Toys and DIY
8,Pede Ultrices Ind...,([computer progra...,10279061213,computer programm...,2,"Beauty, Health, P..."
9,Nunc Inc.,"[(furniture, home...",10323485998,furniture home fu...,0,Furniture


In [21]:
final_join.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [22]:
# final_join.limit(5)
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import when
from pyspark.sql import functions as countDistinct

In [23]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(F.avg("cons_fraud_prob").\
alias("avg_consumer_fraud"),F.avg("merch_fraud_prob").alias("avg_merchant_fraud"))

In [24]:
aggregated_prob.show()

+--------------------+------------------+------------------+
|                 _c6|avg_consumer_fraud|avg_merchant_fraud|
+--------------------+------------------+------------------+
|                null|15.137285023740146|              null|
|         Electronics| 15.09275324977202|29.632925771641904|
|        Toys and DIY|15.125051928052184|  32.3724283793317|
|           Furniture|15.107745977816467|30.946188701547527|
|Beauty, Health, P...|15.148490844867114|29.759879650543628|
|Books, Stationary...|15.106109093112805|29.020184401115156|
+--------------------+------------------+------------------+



In [25]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085)&(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)

In [26]:
aggregated_prob.limit(6)

_c6,avg_consumer_fraud,avg_merchant_fraud
null,15.137285023740146,null
Electronics,15.09275324977202,29.632925771641904
Toys and DIY,15.125051928052184,32.3724283793317
Furniture,15.107745977816467,30.946188701547527
"Beauty, Health, P...",15.148490844867114,29.759879650543628
"Books, Stationary...",15.106109093112805,29.020184401115156


In [27]:
final_join.limit(5)

merchant_abn,revenue_levels,dollar_value,user_id,consumer_id,postcodes,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6
63290521567,a,55.6054944945403,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY
63290521567,a,55.6054944945403,667,1549,4705,667,2021-10-04,9.674041120495586,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY
24852446429,c,44.50284528982194,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture
24852446429,c,44.50284528982194,667,1549,4705,667,2021-10-04,9.674041120495586,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture
68216911708,c,18.476243072950304,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,2620,Placerat Eget Ven...,"[[computers, comp...",68216911708,computers compute...,4,Electronics


In [28]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)
model_with_fraud = model_with_fraud.withColumnRenamed("merchant_abn","sus_merchant_abn")

In [29]:
model_with_fraud.limit(5)

sus_merchant_abn,revenue_levels,dollar_value,user_id,consumer_id,postcodes,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6,is_fraud
63290521567,a,55.6054944945403,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY,0
63290521567,a,55.6054944945403,667,1549,4705,667,2021-10-04,9.674041120495586,null,null,null,2391,Vehicula Pellente...,((artist supply a...,63290521567,artist supply and...,1,Toys and DIY,0
24852446429,c,44.50284528982194,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture,0
24852446429,c,44.50284528982194,667,1549,4705,667,2021-10-04,9.674041120495586,null,null,null,680,Erat Vitae LLP,[(florists suppli...,24852446429,florists supplies...,0,Furniture,0
68216911708,c,18.476243072950304,667,1549,4705,667,2021-09-09,17.802875300207035,null,null,null,2620,Placerat Eget Ven...,"[[computers, comp...",68216911708,computers compute...,4,Electronics,0


In [30]:
# # model_with_fraud.count()
# merchant_abn_col = model_with_fraud.select("merchant_abn")
# print(merchant_abn_col.count())

In [31]:
# model_with_fraud.write.parquet("../data/curated/model_data_with_fraud_col.parquet") 

In [32]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("sus_merchant_abn").agg(F.avg("is_fraud").alias("average fraud rate per merchant"))

In [33]:

final_model_with_fraud.limit(5)

sus_merchant_abn,average fraud rate per merchant
83412691377,0.0
48214071373,0.0
38700038932,0.0
41956465747,0.0
73256306726,0.0


In [34]:

final_model_with_fraud.limit(20)

non_zero = final_model_with_fraud.filter(final_model_with_fraud["average fraud rate per merchant"]> 0)
non_zero.limit(5)

sus_merchant_abn,average fraud rate per merchant
90568944804,0.1671796450189802
27093785141,0.17402953791635606
19492220327,0.14381233000906618
76968105359,0.24384787472035793
97884414539,0.358695652173913


In [35]:
final_model_with_fraud.write.parquet("../data/tables/transactions_with_fraud_rates.parquet")

AnalysisException: path file:/Users/ahirve/Desktop/ads_bnpl/data/tables/transactions_with_fraud_rates.parquet already exists.

In [ ]:
#merging with full dataset with all columns for geospatial visualisations 
full_data
final_model_with_fraud

full_data.createOrReplaceTempView("temp")

final_model_with_fraud.createOrReplaceTempView("temp2")

full_data_with_fraud = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.sus_merchant_abn
""")

full_data_with_fraud.write.parquet('../data/tables/full_dataset_with_fraud_rates.parquet')

In [ ]:
# data = spark.read.parquet('../data/tables/full_join.parquet')

In [ ]:
# data.limit(5)